## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
pip install gmaps

Note: you may need to restart the kernel to use updated packages.


In [2]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps
import numpy as np
# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [3]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("/Users/mikeschnur/Desktop/Bootcamp Folder/World_Weather_Analysis/Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Weiser,US,44.2510,-116.9693,69.31,27,0,2.04,clear sky
1,1,Cabo San Lucas,MX,22.8909,-109.9124,87.19,61,20,14.97,few clouds
2,2,Azacualpa,HN,15.3433,-88.5514,84.85,59,51,3.38,broken clouds
3,3,Atambua,ID,-9.1061,124.8925,76.05,84,96,1.28,overcast clouds
4,4,Cape Town,ZA,-33.9258,18.4232,65.12,88,75,13.80,broken clouds


In [4]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 85


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
pref_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
pref_cities_df.head(10)

,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Azacualpa,HN,15.3433,-88.5514,84.85,59,51,3.38,broken clouds
3,3,Atambua,ID,-9.1061,124.8925,76.05,84,96,1.28,overcast clouds
14,14,Mahebourg,MU,-20.4081,57.7000,70.09,73,40,9.22,scattered clouds
15,15,Avarua,CK,-21.2078,-159.7750,75.25,69,40,14.97,scattered clouds
16,16,Hambantota,LK,6.1241,81.1185,77.38,85,99,17.40,overcast clouds
19,19,Impfondo,CG,1.6381,18.0667,77.70,78,76,0.89,broken clouds
31,31,Vila Velha,BR,-20.3297,-40.2925,78.75,79,40,14.97,scattered clouds
34,34,Georgetown,MY,5.4112,100.3354,81.23,90,20,0.00,light rain
36,36,Bambous Virieux,MU,-20.3428,57.7575,70.05,73,40,9.22,scattered clouds
40,40,Anloga,GH,5.7947,0.8973,80.10,81,67,8.77,broken clouds


In [6]:
# 4a. Determine if there are any empty rows.
print(len(pref_cities_df))
pref_cities_df.count()

252


CityID                 252
City                   252
Country                248
Lat                    252
Lng                    252
Max Temp               252
Humidity               252
Cloudiness             252
Wind Speed             252
Current Description    252
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = pref_cities_df.dropna(axis="index", how="any")
print(len(clean_df))
clean_df.head()

248


,CityID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
2,2,Azacualpa,HN,15.3433,-88.5514,84.85,59,51,3.38,broken clouds
3,3,Atambua,ID,-9.1061,124.8925,76.05,84,96,1.28,overcast clouds
14,14,Mahebourg,MU,-20.4081,57.7000,70.09,73,40,9.22,scattered clouds
15,15,Avarua,CK,-21.2078,-159.7750,75.25,69,40,14.97,scattered clouds
16,16,Hambantota,LK,6.1241,81.1185,77.38,85,99,17.40,overcast clouds


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Azacualpa,HN,84.85,broken clouds,15.3433,-88.5514,
3,Atambua,ID,76.05,overcast clouds,-9.1061,124.8925,
14,Mahebourg,MU,70.09,scattered clouds,-20.4081,57.7000,
15,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,
16,Hambantota,LK,77.38,overcast clouds,6.1241,81.1185,
19,Impfondo,CG,77.70,broken clouds,1.6381,18.0667,
31,Vila Velha,BR,78.75,scattered clouds,-20.3297,-40.2925,
34,Georgetown,MY,81.23,light rain,5.4112,100.3354,
36,Bambous Virieux,MU,70.05,scattered clouds,-20.3428,57.7575,
40,Anloga,GH,80.10,broken clouds,5.7947,0.8973,


In [9]:
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
hotels = requests.get(base_url, params=params).json()
hotels

{'html_attributions': [], 'results': [], 'status': 'INVALID_REQUEST'}

In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row['Lat']
    lng = row['Lng']
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        print("Success")
    except (IndexError):
        print("Hotel not found... skipping.")
        

Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Hotel not found... skipping.
Success
Hotel not found... skipping.
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Success
Success
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Hotel not found... skipping.
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success
Success


In [16]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]  
clean_hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
2,Azacualpa,HN,84.85,broken clouds,15.3433,-88.5514,Hotel Merendon
3,Atambua,ID,76.05,overcast clouds,-9.1061,124.8925,Ros Muskanan
14,Mahebourg,MU,70.09,scattered clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
15,Avarua,CK,75.25,scattered clouds,-21.2078,-159.7750,Paradise Inn
16,Hambantota,LK,77.38,overcast clouds,6.1241,81.1185,Bungalow 63


In [17]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [18]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
figure = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
figure.add_layer(heat_layer)
figure.add_layer(marker_layer)
# 11b. Display the figure
figure

Figure(layout=FigureLayout(height='420px'))